# Demo: Watershed on microglia 

### Get the watershed_3d package, version 0.0.2

In [1]:
!pip uninstall -y watershed_3d
!pip install --no-cache-dir  https://github.com/acycliq/3d_watershed/raw/master/dist/watershed_3d-0.0.4-py3-none-any.whl

Found existing installation: watershed-3d 0.0.4
Uninstalling watershed-3d-0.0.4:
  Successfully uninstalled watershed-3d-0.0.4


### Import some other packages needed for this notebook

In [2]:
import watershed_3d as ws
import pandas as pd
import numpy as np
from scipy.sparse import coo_matrix
from PIL import Image, ImageDraw

DIPlib -- a quantitative image analysis library
Version 3.4.1 (Oct 13 2023)
For more information see https://diplib.org


In [3]:
print(ws.__version__)

0.0.4


### Do now the segmentation. Call the app method and pass in the path to the 3d-image and a list of the pages that havent been imaged properly

In [4]:
img_path = r"./microglia_ WT997_icvAB_Iba1_retiled.tif"
bad_pages = list(range(0, 16)) + [65, 66]

Internally the app() method does two things. First, it tries to adjust the brightness of the image and then segments it with watershed. There is a third arg in the app() method which is optional called 'do_rolling_ball' and by default is set to False. If there is an area of the image that is hugely overexposed compared to the rest and the adjustments carried out inside the app() method do not produce a good result, then maybe re-running app() with 'do_rolling_ball' = True would yield better results. However the algorithm will take a lot longer to finish, probably hours.

During the run, the app() method will save some images on the disk, look at the log messages about the exact location. It would be advisable to have a look at these images to get an idea about how well these adjustment work

The 'image_url' that is passed-in to app() should point to a 3d image that is in the format ZYX, ie numStacks-Height-Width 

In [ ]:
## Pass the path to your image as the first argument and then a dictionary
## with any of the following key/values pairs:
# opts = {
#     'exclude_pages': [],    a list with the planes to ignore. Default is an empty list
#     'mode': '2d_stitch',    either '2d_stitch' or '3d'
#     'stitch_threshold': 0.009, If the overlap is bigger than 'stitch_threshold' then the label 
#                                will be passed to the overlapping shape in the following plane.
#                                Default value is 0.009.
#                                This option is relavant only if 'mode' is '2d_stitch' otherwise
#                                it is ignored.
#     'do_rolling_ball': False, If True then the rolling ball filter will be applied on the image. 
#                               Usefule only in cases where locally there is very high brightness 
#                               and the rest of the image or too dark. Default value is False
#     'maxDepth': 1.0           parameter that controls whether two shapes when their boundaries meet, 
#                               will be merged or not. Is it is set too high then you will end up with an 
#                               undersegmented image (few and very large shapes). If it is too low, then 
#                               you will have an oversegmented image (too many and probably small 
#                               shapes). maxDepth is only relevant if mode='3d', otherwise it is ignored.
#    }
#
opts = {
    'exclude_pages': bad_pages,
    'mode': '2d_stitch',
    'stitch_threshold': 0.009,
}
masks = ws.app(image_url=img_path, opts=opts)


2023-11-12 20:59:05,347 - watershed_3d - INFO - Adjusting the image
2023-11-12 21:00:10,995 - watershed_3d - INFO - black and white image saved at ./debug/bw_image.tiff
2023-11-12 21:00:15,770 - watershed_3d - INFO - curve-adjusted image saved at ./debug/adj_img.tiff
2023-11-12 21:00:15,803 - watershed_3d - INFO - Started watershed
2023-11-12 21:07:11,725 - watershed_3d - INFO - Finished watershed
2023-11-12 21:07:11,726 - watershed_3d - INFO - stitching the 2d masks
2023-11-12 21:08:23,546 - watershed_3d - INFO - stitching finished
2023-11-12 21:08:26,510 - watershed_3d - INFO - stitched_masks saved at ./debug/stitched_masks.npy


You can have a preview of the performance of watershed by looking at the images saved in ./debug/segmentation_samples

In [ ]:
# lets see what the max label is
masks.max()

In [ ]:
# lets see now how many distinct labels we have (this will take some seconds...)
np.unique(masks).shape

So, the number of distinct labels is greater than the max number by one? 
Yes, because the number 203975 includes 0, the background.
Hence these two numbers make sense and the microglia shapes have been labelled in 
sequential manner, there are no gaps in the labels

Lets now try to assign the spots to these microglia shapes

## Spot assignement

In [ ]:
spots = pd.read_csv('spots_WT997_icvAB_OMP_restitched.csv')
spots

To assign the spots, call the label_spots() method and pass in the spots dataframe and the masks array that you produced a few steps above from watershed.

The spots dataframe needs to have at least three columns with names 'x', 'y' and 'z_stack'. The label_spots() method will then append and extra column with the name 'label' at the end of this dataframe. It shows which shape the spots has been assigned to. If label=0 then the spot is on the background

In [ ]:
spots_labeled = ws.label_spots(spots, masks)
spots_labeled

## Sanity checking

Lets now play around with one mask and do some sanity checking. This section will also help us to get a better grasp of the data and what they mean.

Pick up the mask that corresponds to the 20th image in the 3d stack

In [ ]:
mask_20 = masks[20]

mask_20 is an array with a lot of zeros (that correspond to the background) and some integer values (labels) that correspond to the microglia shapes.

The lines in the cell below: <br>
    &emsp; 1. convert the array to a boolean, true if the pixel is in microglia, false otherwise <br>
    &emsp;2. convert the boolean to binary (255 if microglia, 0 otherwise)<br>
    &emsp;3. Changes the type to unit8 (necessary to to the image)<br>
    &emsp;4. Does the image from the array

In [ ]:
m = mask_20 > 0
m = 255 * m
m = m.astype(np.uint8)
Image.fromarray(m)

The image above is a black and white image of the segmented 20th page (frame) of the 3d-image

Les now see which shapes are the biggest

In [ ]:
uid, cts = np.unique(mask_20, return_counts=True) 

In [ ]:
df = pd.DataFrame({'label':uid, 'area': cts}).sort_values(by=['area'], ascending=False)
df

The dataframe above shows the area, in pixels, that each shape covers. As expected the background (label=0) covers a big part of the image.

But how about the shape with label=6779. Can we see which one it is?

Yes we can do this by the following code:

In [ ]:
t = mask_20 == 6779 # True if label=6779, False otherwise
t = t.astype(np.uint8)
t = t * 255
Image.fromarray(t.astype(np.uint8))

Ah, dissapointing! It is one of the funny ones at the edges of the image. In fact quite a few of the shapes in the top of the dataframe above are like that; Shapes at the edges

Ok, lets look deeper into that dataframe, lower than the top5

In [ ]:
df.head(20)

Lets pick up label 21471

In [ ]:
t = mask_20 == 21471
t = t.astype(np.uint8)
t = t * 255
microglia_21471 = Image.fromarray(t.astype(np.uint8))
microglia_21471

That looks to be a nice typical miroglia shape!

But how about the spots. Has the label_spots() method (a few steps earlier) done a good job and assigned them properly? 

Lets select the spots that have been assigned to microglia with label 21471 then

In [ ]:
idx = spots_labeled.label == 21471
spots_21471 = spots_labeled[idx]
spots_21471

180 points in total. We will now overlay those spots on the microglia

In [ ]:
# Create a new transparent image of the same size as the original image
overlay = Image.new("RGBA", microglia_21471.size, (255, 255, 255, 0))
draw = ImageDraw.Draw(overlay)#

# Define a color for the random points (red in this case)
point_color = (255, 0, 0, 255)

# draw now the cirles and paint them red
for index, row in spots_21471.iterrows():
    point_size = 5
    x0, y0 = row['x'] - point_size, row['y'] - point_size
    x1, y1 = row['x'] + point_size, row['y'] + point_size
    draw.ellipse([x0, y0, x1, y1], fill=point_color)

# overlay now the dots onto the microglia
combined_img = Image.alpha_composite(microglia_21471.convert("RGBA"), overlay)
combined_img

Hmm, that looks a bit puzzling. The spots look to be clustered on the correct area but not exactly. Clearly there are spots that are outside the boundaries, why is that??

Because the dataframe above (the one with name 'spots_21471') has all the spots across the full z-stack. We are looking at mask 20 here but we have plotted the spots for all Zs

Ah ok, so lets focus on the spots for z=20 then!

In [ ]:
# Christina, i need to explain the logic behind the line below!
idx = np.floor(spots_21471.z_stack == 20) > 0

In [ ]:
# this dataframe has the spots assigned to microglia 21471 and 
# they are on the 20th location in the z-stack
spots_21471_z20 = spots_21471[idx]
spots_21471_z20

Lets try to overlay these points now to microglia_21471. It looks to be only 14 spots

In [ ]:
# Create a new transparent image of the same size as the original image
overlay = Image.new("RGBA", microglia_21471.size, (255, 255, 255, 0))
draw = ImageDraw.Draw(overlay)#

# Define a color for the random points (red in this case)
point_color = (255, 0, 0, 255)

# draw now the cirles and paint them red
for index, row in spots_21471_z20.iterrows():
    point_size = 3
    x0, y0 = row['x'] - point_size, row['y'] - point_size
    x1, y1 = row['x'] + point_size, row['y'] + point_size
    draw.ellipse([x0, y0, x1, y1], fill=point_color)

# overlay now the dots onto the microglia
combined_img = Image.alpha_composite(microglia_21471.convert("RGBA"), overlay)
combined_img

That looks better! Can we have a closer look.

Ok, lets crop the image then. We need to get some idea of the area to crop

In [ ]:
padding = 10

In [ ]:
xmin = spots_21471.x.min() - padding
ymin = spots_21471.y.min() - padding
xmax = spots_21471.x.max() + padding
ymax = spots_21471.y.max() + padding

Lets crop the image.

In [ ]:
crop = combined_img.crop((xmin, ymin, xmax, ymax))
crop

It doesnt look that bad, Actually, it makes sense. All spots are within the shape. It seems that the label_spots() function works as expected